In [12]:
HOPSWORKS_PROJECT_NAME = 'nyc_taxiride_demand'

In [13]:
import sys
sys.path.append('../')

import os
from dotenv import load_dotenv
from src.paths import PARENT_DIR

# load key-value pairs from .env file located in the parent directory
load_dotenv(PARENT_DIR / '.env')

HOPSWORKS_API_KEY = os.environ['HOPSWORKS_API_KEY']

In [14]:
from datetime import datetime
import pandas as pd
from src.data import load_raw_data

from_year = 2024
to_year = datetime.now().year
print(f"Downloading raw data from {from_year} to {to_year}")

rides = pd.DataFrame()
for year in range(from_year, to_year + 1):
    # download data for the whole year
    rides_one_year = load_raw_data(year)

    # append rows
    rides = pd.concat([rides, rides_one_year])

File 2024-01 was already in local storage
File 2024-02 was already in local storage
File 2024-03 was already in local storage
File 2024-04 was already in local storage
File 2024-05 was already in local storage
File 2024-06 was already in local storage
File 2024-07 was already in local storage
File 2024-08 was already in local storage
File 2024-09 was already in local storage
File 2024-10 was already in local storage
File 2024-11 was already in local storage
File 2024-12 was already in local storage
File 2025-01 was already in local storage
File 2025-02 was already in local storage
File 2025-03 was already in local storage
File 2025-04 was already in local storage
File 2025-05 was already in local storage
File 2025-06 was already in local storage
File 2025-07 was already in local storage
File 2025-08 was already in local storage
File 2025-09 was already in local storage
File 2025-10 was already in local storage
File 2025-11 was already in local storage
2025-12 file is not available


In [15]:
print(f"{len(rides) = }")

len(rides) = 85586696


In [16]:
from src.data import transform_raw_data_into_ts_data

ts_data = transform_raw_data_into_ts_data(rides)

100%|██████████| 263/263 [00:12<00:00, 21.46it/s]


In [17]:
import hopsworks

In [18]:
project = hopsworks.login(
    project=HOPSWORKS_PROJECT_NAME,
    api_key_value=HOPSWORKS_API_KEY
)

2025-12-31 10:36:21,662 INFO: Closing external client and cleaning up certificates.
2025-12-31 10:36:21,688 INFO: Connection closed.
2025-12-31 10:36:21,699 INFO: Initializing external client
2025-12-31 10:36:21,703 INFO: Base URL: https://c.app.hopsworks.ai:443




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'


2025-12-31 10:36:22,413 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1329302


In [19]:
feature_store = project.get_feature_store()

In [20]:
# To save data into feature store use feature groups api to write data.
# This needs feature group name and version
FEATURE_GROUP_NAME = 'time_series_hourly_feature_group'
FEATURE_GROUP_VERSION = 1

In [21]:
# Create feature group
feature_group = feature_store.get_or_create_feature_group(
    name=FEATURE_GROUP_NAME,
    version=FEATURE_GROUP_VERSION,
    description="Time-series data at hourly frequency",
    primary_key=['pickup_location_id', 'pickup_hour'], # A unique identifier for each row in the data (location_id and hour)
    event_time='pickup_hour' # This is the timestamp behind those events
)

In [22]:

feature_group.insert(ts_data, 
                     write_options={"wait_for_job": False}) # Don't wait for this job to finalize.

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1329302/fs/1317957/fg/1876549


Uploading Dataframe: 100.00% |██████████| Rows 4418400/4418400 | Elapsed Time: 04:46 | Remaining Time: 00:00


Launching job: time_series_hourly_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1329302/jobs/named/time_series_hourly_feature_group_1_offline_fg_materialization/executions


(Job('time_series_hourly_feature_group_1_offline_fg_materialization', 'SPARK'),
 None)